<a href="https://colab.research.google.com/github/echoqshen/AWS_RDS/blob/main/sean_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!wget -q http://www.java2s.com/Code/JarDownload/mysql/mysql-connector-java-5.1.17-bin.jar.zip
!tar xf spark-3.2.0-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pymysql
!pip install -q sqlalchemy
!pip install -q mysql
!pip install mysql-connector-python-rf

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
#database-2.cassuiwt6cgh.us-east-1.rds.amazonaws.com
#postgres
#password
#5432



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-18 04:34:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2022-01-18 04:34:17 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sean_test").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://seanstest2.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df=spark.read.csv(SparkFiles.get('user_data.csv'), sep=",", header=True)
user_data_df.show()

+---+----------+---------+-----------+--------------------+--------------------+------------+
| id|first_name|last_name|active_user|      street_address|               state|    username|
+---+----------+---------+-----------+--------------------+--------------------+------------+
|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|   ibearham0|
|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|    wwaller1|
|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|   ichesnut2|
|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|     tsnarr3|
|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|   fwherrit4|
|  6|    Fraser|  Korneev|       TRUE|  76084 Novick Court|           Minnesota|  fstappard5|
|  7|    Demott|   Rapson|       TRUE|    86320 Dahle Park|District of Columbia|  lhambling6|
|  8|    Robert|    Poile|      FALSE|1540 Manitowish Hill| 

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://seanstest2.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df=spark.read.csv(SparkFiles.get('user_payment.csv'), sep=",", header=True)
user_payment_df.show(5)



+----------+---------+--------------------+
|billing_id| username|        cc_encrypted|
+----------+---------+--------------------+
|         1|ibearham0|a799fcafe47d7fb19...|
|         2| wwaller1|a799fcafe47d7fb19...|
|         3|ichesnut2|a799fcafe47d7fb19...|
|         4|  tsnarr3|a799fcafe47d7fb19...|
|         5|fwherrit4|a799fcafe47d7fb19...|
+----------+---------+--------------------+
only showing top 5 rows



In [ ]:
joined_df = user_data_df.join(user_payment_df, on='username', how='inner')
joined_df.show(10)

+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  ibearham0|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|   wwaller1|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|  ichesnut2|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|    tsnarr3|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|  fwherrit4|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7fb19...|


In [ ]:
dropna_df = joined_df.dropna()
dropna_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [ ]:
from pyspark.sql.functions import col

In [ ]:
cleaned_df = dropna_df.filter(col('active_user')==True)
cleaned_df.show()


+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|     username| id|first_name|  last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|   fstappard5|  6|    Fraser|    Korneev|       TRUE|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|   lhambling6|  7|    Demott|     Rapson|       TRUE|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|    wheinerte| 15|   Sadella|      Jaram|       TRUE|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
| droughsedgeg| 17|    Hewitt|    Trammel|       TRUE|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|    ydudeniei| 19|       Ted|    Knowlys|       TRUE|      31 South Drive|                Ohio| 

In [ ]:
clean_user_df = cleaned_df.select(['id', 'first_name', 'last_name','username'])
clean_user_df.show(5)
# Write DataFrame to active_user table in RDS
#clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

NameError: ignored

In [ ]:
# Store environmental variable
from getpass import getpass
#password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cxvwccsivezo.us-east-1.rds.amazonaws.com/postgres"
config = {"user":"postgres",
          "password": "Dayton005",
          "driver":"org.postgresql.Driver"}
          

In [ ]:
# Write DataFrame to active_user table in RDS
data = [['tom', 10], ['nick', 15], ['juli', 14]]
import pandas as pd
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Name', 'Age'])
df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

AttributeError: ignored